In [1]:
# Cell 1: Import required libraries
import pandas as pd
import numpy as np
import sqlite3
import time
import datetime
from icoscp.cpb.dobj import Dobj
import os
from icoscp_core.icos import bootstrap
from icoscp import cpauth

/tmp/ipykernel_16334/3369797552.py:7: UserWarning: 
It is highly recommended to replace the following import:
"from icoscp.cpb.dobj import Dobj"
  with
"from icoscp.dobj import Dobj"
Find out more here: https://icos-carbon-portal.github.io/pylib/icoscp/install/#upgrade-guide
  from icoscp.cpb.dobj import Dobj


In [2]:
# Cell 2: Input parameters (edit these)
OSVAS='/home/pn56/OSVASgh/'
station_name = "Majadas_south"
start_date = "2016-06-01"
end_date = "2016-07-31"
station_list_path = os.path.join(OSVAS,"./sqlites/station_list_SURFEX.csv")
dataset_doi = "https://meta.icos-cp.eu/objects/dDlpnhS3XKyZjB22MUzP_nAm"

#In the following dictionary, select the variable names from dataset_doi 
#and how they will be renamed in the output sqlite

variables = {'H_F_MDS': 'H', 'LE_F_MDS': 'LE'}

#Authenticate into ICOS:
cookie_file_path = os.path.join(OSVAS,"./sqlites/icos_cookie.txt")
cookie_token=open(cookie_file_path,'r').readline().rstrip()
meta, data = bootstrap.fromCookieToken(cookie_token)
cpauth.init_by(data.auth)

#Test: If the authentication went well, these lines of code will not fail:
import icoscp
from icoscp.dobj import Dobj
obj_flux='https://meta.icos-cp.eu/objects/dDlpnhS3XKyZjB22MUzP_nAm'
dobj_flux=Dobj(obj_flux).data

In [3]:
# Cell 3: Define helper functions
def load_station_metadata(path):
    return pd.read_csv(path)

def get_station_info(name, metadata_df):
    row = metadata_df[metadata_df['name'] == name]
    if row.empty:
        raise ValueError(f"Station '{name}' not found in the metadata file.")
    return row.iloc[0]

def fetch_flux_data(doi, cookie_token):
    # Bootstrap session using your token
    meta, data = bootstrap.fromCookieToken(cookie_token)
    cpauth.init_by(data.auth)

    # Fetch the data object
    dobj = Dobj(doi)
    df = dobj.data
    return df


def fetch_flux_data(doi):
    dobj = Dobj(doi)
    df = dobj.data
    return df


def process_data(df, station_info, start, end):
    df['valid_dttm'] = pd.to_datetime(df['TIMESTAMP'], utc=True)
    df = df[(df['valid_dttm'] >= start) & (df['valid_dttm'] <= end)]
    df['valid_dttm'] = df['TIMESTAMP'].view('int64')
    df = df.dropna(subset=['H_F_MDS', 'LE_F_MDS'])
    df['SID'] = station_info['SID']
    df['lat'] = station_info['lat']
    df['lon'] = station_info['lon']
    df['elev'] = station_info['elev']
    return df[['valid_dttm', 'SID', 'lat', 'lon', 'elev', 'H_F_MDS', 'LE_F_MDS']]

def process_data(df, station_info, start, end, variable_names):
    """
    Processes a DataFrame, selecting data within a time range, adding station information,
    and renaming specified columns based on a dictionary.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'TIMESTAMP' column and the variables to process.
        station_info (dict): A dictionary containing station information with keys 'SID', 'lat', 'lon', and 'elev'.
        start (str or datetime): The start timestamp for filtering.
        end (str or datetime): The end timestamp for filtering.
        variable_names (dict): A dictionary where keys are the original column names in df
                               and values are the desired new column names.

    Returns:
        pd.DataFrame: A DataFrame containing the processed data with renamed columns.
    """
    df['valid_dttm'] = pd.to_datetime(df['TIMESTAMP'], utc=True)
    df = df[(df['valid_dttm'] >= start) & (df['valid_dttm'] <= end)].copy()
    df['valid_dttm'] = df['TIMESTAMP'].view('int64')

    # Select and drop rows with NaN values for the specified variables
    variables_to_process = list(variable_names.keys())
    df = df.dropna(subset=variables_to_process).copy()

    df['SID'] = station_info['SID']
    df['lat'] = station_info['lat']
    df['lon'] = station_info['lon']
    df['elev'] = station_info['elev']

    # Select the desired columns and rename them
    columns_to_select = ['valid_dttm', 'SID', 'lat', 'lon', 'elev'] + variables_to_process
    df_processed = df[columns_to_select].rename(columns=variable_names)

    return df_processed

In [4]:
# Cell 4: Load metadata and fetch data
station_metadata = load_station_metadata(station_list_path)
station_info = get_station_info(station_name, station_metadata)
print(f"Loaded metadata for station: {station_name}")
print(station_info)
df_raw = fetch_flux_data(dataset_doi)
#df_raw = fetch_flux_data(dataset_doi, cookie_file_path)
print("Fetched data from ICOS.")
df_raw.head()

Loaded metadata for station: Majadas_south
SID        4300000005
lat         39.940556
lon         -5.774722
elev              258
name    Majadas_south
Name: 4, dtype: object
Fetched data from ICOS.


,GPP_DT_VUT_REF,GPP_NT_VUT_REF,H_F_MDS,H_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LW_IN_F,LW_IN_F_QC,NEE_VUT_REF,NEE_VUT_REF_QC,...,SW_IN_F,SW_IN_F_QC,TA_F,TA_F_QC,TIMESTAMP,TIMESTAMP_END,VPD_F,VPD_F_QC,WS_F,WS_F_QC
0,NaN,NaN,NaN,3,NaN,3,NaN,2,NaN,3,...,NaN,2,NaN,2,2013-12-31 23:30:00,2014-01-01 00:30:00,NaN,2,NaN,2
1,NaN,NaN,NaN,3,NaN,3,NaN,2,NaN,3,...,NaN,2,NaN,2,2014-01-01 00:00:00,2014-01-01 01:00:00,NaN,2,NaN,2
2,NaN,NaN,NaN,3,NaN,3,NaN,2,NaN,3,...,NaN,2,NaN,2,2014-01-01 00:30:00,2014-01-01 01:30:00,NaN,2,NaN,2
3,NaN,NaN,NaN,3,NaN,3,NaN,2,NaN,3,...,NaN,2,NaN,2,2014-01-01 01:00:00,2014-01-01 02:00:00,NaN,2,NaN,2
4,NaN,NaN,NaN,3,NaN,3,NaN,2,NaN,3,...,NaN,2,NaN,2,2014-01-01 01:30:00,2014-01-01 02:30:00,NaN,2,NaN,2


In [5]:
# Cell 5: Process and preview final data
df_processed = process_data(df_raw, station_info, start_date, end_date, variables)
df_processed.head()

/tmp/ipykernel_16334/1733085736.py:57: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df['valid_dttm'] = df['TIMESTAMP'].view('int64')


,valid_dttm,SID,lat,lon,elev,H,LE
42337,1464739200000,4300000005,39.940556,-5.774722,258,-7.900000,4.22300
42338,1464741000000,4300000005,39.940556,-5.774722,258,-16.196301,-1.55100
42339,1464742800000,4300000005,39.940556,-5.774722,258,-7.753000,2.78588
42340,1464744600000,4300000005,39.940556,-5.774722,258,-11.108000,2.74053
42341,1464746400000,4300000005,39.940556,-5.774722,258,-13.969000,4.66500


In [11]:
# Cell 6: Write to SQLite
year = pd.to_datetime(start_date).year
output_dir = os.path.join(OSVAS,"SYNOP",station_name)
os.makedirs(output_dir, exist_ok=True) # Create directory if it doesn't exist
output_file = os.path.join(output_dir, f"OBSTABLE_{year}.sqlite")


with sqlite3.connect(output_file) as conn:
    df_processed.to_sql("SYNOP", conn, if_exists="replace", index=False)

    conn.execute("DROP TABLE IF EXISTS tmp")
    conn.execute("""
        CREATE TABLE tmp (
            valid_dttm REAL,
            SID REAL,
            lat REAL,
            lon REAL,
            elev REAL,
            H REAL,
            LE REAL
        )
    """)
    conn.execute("INSERT INTO tmp SELECT * FROM observations")
    conn.execute("DROP TABLE observations")
    conn.execute("ALTER TABLE tmp RENAME TO observations")

print(f"Data written to {output_file}")

Data written to /home/pn56/OSVASgh/observations/Majadas_south/OBSTABLE_2016.sqlite
